Pasos:

Extraer con cpptraj las trayectorias con únicamente los CA y proteina

% Ya en eel nootebook de pcas

PAra cada ensayo:

- Carga la conformación inicial
- Carga las trayectorias (5 de 20 ns)
- para PCA
    - Alinear con respecto a la inicial (la estructura cristalográfica inicial)
         - la idea es que la incial es un aestructura cristalográfica y está proyección original
    - Alinear todas las conformaciones de la producción, de tal manera que el clustering y el PCA permita extraer las conformaciones centroides de únicamente este espacio
    - Usar los residuos definidos por Pisani
    - Usar los residuos de la estructura secundaria
    - Usar los residuos del pocket
- Proyección con MDS
    - Las estruturas se alinean con respecto al frame 1 de las cristalográficas
    - Revisar el notebook proyeccion, dejamos pendiente por ahora esta parte
 

In [48]:
import subprocess
import glob
import os
from pathlib import Path

In [ ]:
DM_DIR = os.path.join('..', '..', 'ARCHIVOS', 'DINAMICA')

In [58]:
# Cambio de nombre de todos prmtop
for file_path in Path(DM_DIR).glob('**/*.parm7'):
    file_path.rename(file_path.with_suffix('.prmtop'))


In [23]:


def strip_traj_cpptraj(system_prod_dir, file_system_name, run = True):
    if not run:
        return(None)
        
    wd_dir = os.getcwd()
    # Crea el archivo de entrada para ejecutar cpptraj
    with open(F'{system_prod_dir}/run_strip.cpptraj', 'w') as f:
        f.write(F'''# Strip CA atoms from traj
parm ../{file_system_name}.prmtop
trajin {file_system_name}_prod.nc
reference ../{file_system_name}.rst7
strip :WAT:Cl-:Na+:ETA outprefix PROT
trajout ./PROT.{file_system_name}_prod.nc
go
strip !(@CA) outprefix CA
trajout ./CA.{file_system_name}_prod.nc
go
    ''') # NOTA: de ser necesario, actualizar moléculas del solvente
    # Se dirije al directorio de producción para ejecutar el archivo
    os.chdir(system_prod_dir)
    try:
        os.system('cpptraj -i run_strip.cpptraj')
    except ValueError as e:
        print('Hubo un error en la ejecución', e)
    finally:
        os.chdir(wd_dir)

In [34]:
def process_trajs(system, md_type, prod_dir, system_name, run = True):
    prod_files = glob.glob(os.path.join(DM_DIR, system, md_type, 'rep_*', prod_dir))
    for system_prod_dir in prod_files:
        print(F'Procesando {system_prod_dir}')
        strip_traj_cpptraj( system_prod_dir, system_name, run = run)

## Directorio general de las dinámicas

In [4]:
import glob
DM_DIR = os.path.join('..', '..', 'ARCHIVOS', 'DINAMICA')

# Sistema 4fkw


> ## MD / Simulated Annealing / apo

In [37]:
system = '4fku'
md_type = 'sa_dm'
prod_dir = '3_prod'
system_name = 'cdk2'
# Tenemos cinco réplicas
process_trajs(system, md_type, prod_dir, system_name, run = True)

Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_dm/rep_2/3_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_dm/rep_1/3_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_dm/rep_3/3_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_dm/rep_4/3_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_dm/rep_5/3_prod


> ## Solventes Mixtos / Plain MD

In [44]:
system = '4fku'
md_type = 'mix_solv'
prod_dir = '4_prod'
system_name = 'mxs_cdk2'
# Tenemos cinco réplicas
process_trajs(system, md_type, prod_dir, system_name, run = True)

Procesando ../../ARCHIVOS/DINAMICA/4fku/mix_solv/rep_2/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/mix_solv/rep_1/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/mix_solv/rep_3/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/mix_solv/rep_4/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/mix_solv/rep_5/4_prod


> ## Solventes mixtos / Simulated Annealing

In [35]:
system = '4fku'
md_type = 'mxs_sa'
prod_dir = '4_prod'
system_name = 'mxs_cdk2'
# Tenemos cinco réplicas
process_trajs(system, md_type, prod_dir, system_name, run = False)

Procesando ../../ARCHIVOS/DINAMICA/4fku/mxs_sa/rep_2/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/mxs_sa/rep_1/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/mxs_sa/rep_3/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/mxs_sa/rep_4/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/mxs_sa/rep_5/4_prod


> ## Simulated Annealing / PL complex (CS62 lig)

In [42]:
system = '4fku'
md_type = 'sa_lig_CS262'
prod_dir = '4_prod'
system_name = 'cdk2_LIG'
# Tenemos cinco réplicas
process_trajs(system, md_type, prod_dir, system_name, run = True)

Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_lig_CS262/rep_2/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_lig_CS262/rep_1/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_lig_CS262/rep_3/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_lig_CS262/rep_4/4_prod
Procesando ../../ARCHIVOS/DINAMICA/4fku/sa_lig_CS262/rep_5/4_prod


# Sistema 1fin
> ## MD / simulated Annealing

In [38]:
system = '1fin'
md_type = 'sa_dm'
prod_dir = '4_prod'
system_name = 'sa_dm_1fin'
# Tenemos cinco réplicas
process_trajs(system, md_type, prod_dir, system_name, run = True)

Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_dm/rep_2/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_dm/rep_1/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_dm/rep_3/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_dm/rep_4/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_dm/rep_5/4_prod


In [39]:
system = '1fin'
md_type = 'mxs_sa'
prod_dir = '4_prod'
system_name = 'mxs_1fin'
# Tenemos cinco réplicas
process_trajs(system, md_type, prod_dir, system_name, run = True)

Procesando ../../ARCHIVOS/DINAMICA/1fin/mxs_sa/rep_2/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/mxs_sa/rep_1/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/mxs_sa/rep_3/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/mxs_sa/rep_4/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/mxs_sa/rep_5/4_prod


In [40]:
system = '1fin'
md_type = 'sa_lig_ATP'
prod_dir = '4_prod'
system_name = 'sa_lig_1fin_ATP'
# Tenemos cinco réplicas
process_trajs(system, md_type, prod_dir, system_name, run = True)

Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_ATP/rep_2/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_ATP/rep_1/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_ATP/rep_3/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_ATP/rep_4/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_ATP/rep_5/4_prod


In [43]:
system = '1fin'
md_type = 'sa_lig_CS262'
prod_dir = '4_prod'
system_name = 'sa_lig_1fin_CS262'
# Tenemos cinco réplicas
process_trajs(system, md_type, prod_dir, system_name, run = True)

Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_CS262/rep_2/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_CS262/rep_1/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_CS262/rep_3/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_CS262/rep_4/4_prod
Procesando ../../ARCHIVOS/DINAMICA/1fin/sa_lig_CS262/rep_5/4_prod
